# 1. Motivation of the solution

#### Which has been our initial approach to the problem? 

As Physics students we know that understanding thoroughly what is being asked is crucial for model selection. On the other hand, we think it is very important to assess which variables are relevant for solving the problem. Hence, we spent the morning studying the nature of our problem and converting all training data files to the same format, so we could train our model comfortably.  

#### Which is the nature of the proposed problem? 

As both inputs and outputs are labelled, we deduced that needed to use a Supervised Machine Learning Algorithm. Besides, we realised we were in front of a multi-class classification problem. After revising which algorithm are suitable for that sort of problem, we decided to implement a Random Forest. 

#### Why did we choose the implement a Random Forest algorithm?

As we have never tried to use Machine Learning before, we used the most understandable model for us. Essentially, Random Forest consists of a superposition of decision trees, which are familiar to us since we studied option pricing. Moreover, we were keen on learning about this algorithm since it is widely used both in classification and regression models. 

# 2. Implementation of the solution 

### Relevant packages import

In [1]:
import numpy as np
import pandas as pd
import urllib.request, json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

### Data processsing

In [2]:
#train3 data set , extract information from the url
with urllib.request.urlopen("http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first") as url:
    data3 = json.loads(url.read().decode())
    
#create the json file    
with open('train3.json', 'w') as outputFile:
    json.dump(data3, outputFile, sort_keys= True)

In [3]:
#train4 data set
with urllib.request.urlopen("http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second") as url:
    data4 = json.loads(url.read().decode())
    
#create the json file    
with open('train4.json', 'w') as outputFile:
    json.dump(data4, outputFile, sort_keys= True)   

In [4]:
#train5 data set
with urllib.request.urlopen("http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third") as url:
    data5 = json.loads(url.read().decode())
    
#create the json file    
with open('train5.json', 'w') as outputFile:
    json.dump(data5, outputFile, sort_keys= True)  

To begin with, we processed data fitted the model with one dataset and then replicated the same methodolgy with the others.

In [5]:
#load all the data sets
train1 = pd.read_csv('train1.csv')
train2 = pd.read_csv('train2.csv', sep = ';')
train3 = pd.read_json('train3.json', orient='values')
train4 = pd.read_json('train4.json', orient='values')
train5 = pd.read_json('train5.json', orient='values')

In [6]:
train1.head()

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,Germany,Mineral industry,Installations for the production of cement cli...,https://registry.gdi-de.org/id/de.ni.mu/062217...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,AIR,Carbon dioxide (CO2),2015,10,...,EUROPE,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,2,Mr. Jacob Ortega,7cdb5e74adcb2ffaa21c1b61395a984f
1,Italy,Mineral industry,Installations for the production of cement cli...,IT.CAED/240602021.FACILITY,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,AIR,Nitrogen oxides (NOX),2018,9,...,EUROPE,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,1,Ashlee Serrano,cd1dbabbdba230b828c657a9b19a8963
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/001966000.FACILITY,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,AIR,Methane (CH4),2019,2,...,EUROPE,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,2,Vincent Kemp,5011e3fa1436d15b34f1287f312fbada
3,Czechia,Energy sector,Thermal power stations and other combustion in...,CZ.MZP.U422/CZ34736841.FACILITY,Elektrárny Prunéřov,Kadaň,AIR,Nitrogen oxides (NOX),2012,8,...,EUROPE,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,Carol Gray,37a6d7a71c4f7c2469e4f01b70dd90c2
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,http://paikkatiedot.fi/so/1002031/pf/Productio...,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,AIR,Methane (CH4),2018,12,...,EUROPE,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,2,Blake Ford,471fe554e1c62d1b01cc8e4e5076c61a


In [7]:
#new dataset to work with
df1 = train1.drop('CONTINENT', axis = 1)

In [8]:
df1.head()

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,Germany,Mineral industry,Installations for the production of cement cli...,https://registry.gdi-de.org/id/de.ni.mu/062217...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,AIR,Carbon dioxide (CO2),2015,10,20,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,2,Mr. Jacob Ortega,7cdb5e74adcb2ffaa21c1b61395a984f
1,Italy,Mineral industry,Installations for the production of cement cli...,IT.CAED/240602021.FACILITY,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,AIR,Nitrogen oxides (NOX),2018,9,21,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,1,Ashlee Serrano,cd1dbabbdba230b828c657a9b19a8963
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/001966000.FACILITY,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,AIR,Methane (CH4),2019,2,4,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,2,Vincent Kemp,5011e3fa1436d15b34f1287f312fbada
3,Czechia,Energy sector,Thermal power stations and other combustion in...,CZ.MZP.U422/CZ34736841.FACILITY,Elektrárny Prunéřov,Kadaň,AIR,Nitrogen oxides (NOX),2012,8,6,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,Carol Gray,37a6d7a71c4f7c2469e4f01b70dd90c2
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,http://paikkatiedot.fi/so/1002031/pf/Productio...,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,AIR,Methane (CH4),2018,12,22,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,2,Blake Ford,471fe554e1c62d1b01cc8e4e5076c61a


For working properly `RandomForestClassifier()` inputs need to be `float`. Therefore, we use `LabelEncoder()` to assign a `float` to `string` belongingn to features such as city, for instance.

In [9]:
df1['COUNTRY'] = LabelEncoder().fit_transform(df1['countryName'])
df1['SECTOR'] = LabelEncoder().fit_transform(df1['eprtrSectorName'])
df1['CITY'] = LabelEncoder().fit_transform(df1['CITY ID'])
df1['POLLUTANT'] = LabelEncoder().fit_transform(df1['pollutant'])

In [10]:
#change the values of the 3 pollutants according what is asked for.
df1['POLLUTANT'] = df1['POLLUTANT'].replace([0,1,2],[1,2,0])

Taking into consideration the risk of overfitting our dataset, we removed columns we considered irrelevant for making predicitions. 

In [11]:
#firstly we'll use the first training data set train1, so df1
'''As the training data set 1 and have the same information columns (both are csv), we delete the same columns which
have no relevance in order to find the pollutant, the ones that are string type, we have assigned them an integer'''
#We work only with the first data set and if it works, use the others.

#training data set
x1_train = df1.drop(['POLLUTANT', 'FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'pollutant', 'CITY ID'], axis = 1)

#target variable
y1_train = df1.POLLUTANT

In [12]:
x1_train.head()

,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,COUNTRY,SECTOR,CITY
0,2015,10,20,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,2,10,4,1951
1,2018,9,21,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,1,15,4,3268
2,2019,2,4,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,2,28,8,1248
3,2012,8,6,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,5,2,870
4,2018,12,22,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,2,8,8,1101


In order to asses our model performance, we split data in training and test sets.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(x1_train, y1_train, test_size = 0.10, random_state = 0)

### Model fitting to training data

As the dataset is very big, we increased the number of decision trees as long as accuracy increased.

In [14]:
#Train the randomforest classification model on the training set
#we initially create 500 decision trees
model = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=0)

In [15]:
y_pred = model.predict(X_test)
Y_test = np.array(Y_test)

### Model evaluation 

In [16]:
cm = confusion_matrix(Y_test, y_pred)
print(cm)
#print the accuracy score
accuracy_score(Y_test, y_pred)

[[440 235  50]
 [362 241  70]
 [ 59  24 376]]


0.5691976305869683

We repeated the process showed above for all the datasets available, in search of the one the delivered the highest accuracy.
We might have picked the training and testing data from different datasets or even have mixed them to improve data quality. Nevertheless, we were running out of time and preferred to hand in a simpler but more complete solution.

In [17]:
#do the same for the 2nd training data set and the others
df2 = train2.drop('CONTINENT', axis = 1)
df2['COUNTRY'] = LabelEncoder().fit_transform(df2['countryName'])
df2['SECTOR'] = LabelEncoder().fit_transform(df2['eprtrSectorName'])
df2['CITY'] = LabelEncoder().fit_transform(df2['CITY ID'])
df2['POLLUTANT'] = LabelEncoder().fit_transform(df2['pollutant'])
#in the train2 data set we delete the data from the wind and the temperature because we've had a problem
#with the csv file. It won't be an appropiate dataset to work with
x2_train = df2.drop(['FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'pollutant', 'CITY ID', 'max_wind_speed',
         'avg_wind_speed', 'min_wind_speed','max_temp','avg_temp','min_temp', 'POLLUTANT'], axis = 1)
y2_train = df2.POLLUTANT

In [18]:
x2_train.head()

,reportingYear,MONTH,DAY,DAY WITH FOGS,COUNTRY,SECTOR,CITY
0,2012,4,6,0,10,8,816
1,2007,3,4,1,9,2,1914
2,2008,11,4,1,9,2,855
3,2009,2,12,1,10,8,2066
4,2016,7,21,0,7,2,3122


In [19]:
#we are going to do the same that we've done with the first data set with the 2nd one, which has less information
X2_train, X2_test, Y2_train, Y2_test = train_test_split(x2_train, y2_train, test_size = 0.10, random_state = 0)
model2 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
model2.fit(X2_train, Y2_train)
y2_pred = model2.predict(X2_test)
Y2_test = np.array(Y2_test)
cm2 = confusion_matrix(Y2_test, y2_pred)
print(cm2)
#print the accuracy score
accuracy_score(Y2_test, y2_pred)
#It is unexpected but the accuracy is better than the first training data set

[[374  40 224]
 [ 23 419  34]
 [167  42 534]]


0.7145934302638665

In [20]:
df3 = train3.drop('CONTINENT', axis = 1)
df3['COUNTRY'] = LabelEncoder().fit_transform(df3['countryName'])
df3['SECTOR'] = LabelEncoder().fit_transform(df3['eprtrSectorName'])
df3['CITY'] = LabelEncoder().fit_transform(df3['CITY ID'])
df3['POLLUTANT'] = LabelEncoder().fit_transform(df3['pollutant'])
df3['POLLUTANT'] = df3['POLLUTANT'].replace([0,1,2],[1,2,0])
x3_train = df3.drop(['','FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'pollutant', 'CITY ID', 'EPRTRAnnexIMainActivityCode', 'POLLUTANT'], axis = 1)

y3_train = df3.POLLUTANT
x3_train.head()

,DAY,DAY WITH FOGS,EPRTRSectorCode,MONTH,avg_temp,avg_wind_speed,max_temp,max_wind_speed,min_temp,min_wind_speed,reportingYear,COUNTRY,SECTOR,CITY
0,2,1,4,3,12.326234,15.686450,13.256816,11.019329,14.696895,20.899762,2020,20,1,973
1,3,2,3,2,5.114688,18.396180,4.528859,14.512395,9.219003,23.243403,2019,11,4,3152
2,1,12,1,9,10.208837,21.856210,10.669133,20.262171,14.715465,23.956529,2009,31,2,2668
3,17,1,7,10,9.502302,19.195731,7.095682,18.283547,13.582024,26.696266,2014,15,3,1904
4,6,0,5,9,11.585550,18.277089,9.886774,13.759408,14.006226,24.768933,2015,10,8,3128


In [21]:
#we are going to do the same that we've done with the first data set with the 3rd one
X3_train, X3_test, Y3_train, Y3_test = train_test_split(x3_train, y3_train, test_size = 0.10, random_state = 0)
model3 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
model3.fit(X3_train, Y3_train)
y3_pred = model3.predict(X3_test)
Y3_test = np.array(Y3_test)
cm3 = confusion_matrix(Y3_test, y3_pred)
print(cm3)
#print the accuracy score
accuracy_score(Y3_test, y3_pred)
#It has much more information than the 2nd dataset but the precision is worse

[[234 122  32]
 [218  92  29]
 [ 35  16 172]]


0.5242105263157895

In [22]:
df4 = train3.drop('CONTINENT', axis = 1)
df4['COUNTRY'] = LabelEncoder().fit_transform(df4['countryName'])
df4['SECTOR'] = LabelEncoder().fit_transform(df4['eprtrSectorName'])
df4['CITY'] = LabelEncoder().fit_transform(df4['CITY ID'])
df4['POLLUTANT'] = LabelEncoder().fit_transform(df4['pollutant'])
df4['POLLUTANT'] = df4['POLLUTANT'].replace([0,1,2],[1,2,0])
x4_train = df4.drop(['','FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'pollutant', 'CITY ID', 'EPRTRAnnexIMainActivityCode', 'POLLUTANT'], axis = 1)

y4_train = df4.POLLUTANT
x4_train.head()

,DAY,DAY WITH FOGS,EPRTRSectorCode,MONTH,avg_temp,avg_wind_speed,max_temp,max_wind_speed,min_temp,min_wind_speed,reportingYear,COUNTRY,SECTOR,CITY
0,2,1,4,3,12.326234,15.686450,13.256816,11.019329,14.696895,20.899762,2020,20,1,973
1,3,2,3,2,5.114688,18.396180,4.528859,14.512395,9.219003,23.243403,2019,11,4,3152
2,1,12,1,9,10.208837,21.856210,10.669133,20.262171,14.715465,23.956529,2009,31,2,2668
3,17,1,7,10,9.502302,19.195731,7.095682,18.283547,13.582024,26.696266,2014,15,3,1904
4,6,0,5,9,11.585550,18.277089,9.886774,13.759408,14.006226,24.768933,2015,10,8,3128


In [23]:
#we are going to do the same that we've done with the first data set with the 3rd one
X4_train, X4_test, Y4_train, Y4_test = train_test_split(x4_train, y4_train, test_size = 0.10, random_state = 0)
model4 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
model4.fit(X4_train, Y4_train)
y4_pred = model4.predict(X4_test)
Y4_test = np.array(Y4_test)
cm4 = confusion_matrix(Y4_test, y4_pred)
print(cm4)
#print the accuracy score
accuracy_score(Y4_test, y4_pred)
#It has much more information than the 2nd dataset but the precision is worse

[[234 122  32]
 [218  92  29]
 [ 35  16 172]]


0.5242105263157895

In [24]:
df5 = train3.drop('CONTINENT', axis = 1)
df5['COUNTRY'] = LabelEncoder().fit_transform(df4['countryName'])
df5['SECTOR'] = LabelEncoder().fit_transform(df4['eprtrSectorName'])
df5['CITY'] = LabelEncoder().fit_transform(df4['CITY ID'])
df5['POLLUTANT'] = LabelEncoder().fit_transform(df4['pollutant'])
df5['POLLUTANT'] = df5['POLLUTANT'].replace([0,1,2],[1,2,0])

x5_train = df5.drop(['','FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'pollutant', 'CITY ID', 'EPRTRAnnexIMainActivityCode', 'POLLUTANT'], axis = 1)

y5_train = df5.POLLUTANT
x5_train.head()

,DAY,DAY WITH FOGS,EPRTRSectorCode,MONTH,avg_temp,avg_wind_speed,max_temp,max_wind_speed,min_temp,min_wind_speed,reportingYear,COUNTRY,SECTOR,CITY
0,2,1,4,3,12.326234,15.686450,13.256816,11.019329,14.696895,20.899762,2020,20,1,973
1,3,2,3,2,5.114688,18.396180,4.528859,14.512395,9.219003,23.243403,2019,11,4,3152
2,1,12,1,9,10.208837,21.856210,10.669133,20.262171,14.715465,23.956529,2009,31,2,2668
3,17,1,7,10,9.502302,19.195731,7.095682,18.283547,13.582024,26.696266,2014,15,3,1904
4,6,0,5,9,11.585550,18.277089,9.886774,13.759408,14.006226,24.768933,2015,10,8,3128


In [25]:
#we are going to do the same that we've done with the first data set with the 3rd one
X5_train, X5_test, Y5_train, Y5_test = train_test_split(x5_train, y5_train, test_size = 0.10, random_state = 0)
model5 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
model5.fit(X5_train, Y5_train)
y5_pred = model4.predict(X5_test)
Y5_test = np.array(Y5_test)
cm5 = confusion_matrix(Y5_test, y5_pred)
print(cm5)
#print the accuracy score
accuracy_score(Y5_test, y5_pred)

[[234 122  32]
 [218  92  29]
 [ 35  16 172]]


0.5242105263157895

Eventually, we decided to use the fifth dataset to make our predictions since variables categories mostly coincided with the ones in `test_x.csv`. 

### Data prediction


In [26]:
#test dataset
testdata = pd.read_csv('test_x.csv')
testdata.head()

,test_index,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,0,Poland,3,Mineral industry,3(a),Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,AIR,...,EUROPE,14.080054,14.855940,18.474836,10.278561,11.381181,13.480752,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,1,Luxembourg,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,AIR,...,EUROPE,16.052425,17.623877,22.622900,6.625910,8.840137,13.422924,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,2,Netherlands,1,Energy sector,1(c),Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,AIR,...,EUROPE,13.647318,15.541979,17.818521,5.669436,8.403322,11.276388,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,3,Sweden,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,AIR,...,EUROPE,16.336762,17.458113,19.961777,6.160560,7.571596,9.443572,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,4,Portugal,1,Energy sector,1(c),Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,AIR,...,EUROPE,21.516602,20.532473,21.617137,10.964012,11.548033,12.624114,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7


In order to avoid `ValueErrors`, we need variable's categories and `DataFrame`'s size to coincide.

In [27]:
test = testdata.drop('CONTINENT', axis = 1)
test['COUNTRY'] = LabelEncoder().fit_transform(test['countryName'])
test['SECTOR'] = LabelEncoder().fit_transform(test['eprtrSectorName'])
test['CITY'] = LabelEncoder().fit_transform(test['CITY ID'])

finalx_test = test.drop(['FacilityInspireID', 'City', 'targetRelease', 'REPORTER NAME', 
             'facilityName', 'EPRTRAnnexIMainActivityLabel', 'countryName', 
            'eprtrSectorName', 'CITY ID', 'EPRTRAnnexIMainActivityCode', 'test_index'], axis = 1).dropna()


#we take the same rows the test has in order to be equal in dimension
#finalx_test = np.array(finalx_test)
#finalx_test = finalx_test[:950,:]
#Y5_test = Y5_test[:]
#finalx_test.shape, Y5_test.shape

In [35]:
modelfin = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
modelfin.fit(X5_test, Y5_test)
yfin_pred = modelfin.predict(finalx_test)

In [36]:
#These are the predictions we've made from the test data assuming that the 5th training data set is the best one.
print(yfin_pred)

[2 2 2 ... 2 0 2]


In [37]:
#change the values of the 3 pollutants from the numbers to the elements of pollutant
yfin_pred = pd.DataFrame(yfin_pred)
yfin_pred = yfin_pred.rename(columns={0: 'Pollutant'})
yfin_pred.head()

,Pollutant
0,2
1,2
2,2
3,2
4,0


# 3. Creation of submission files

In [39]:
#csv file using pandas directly
predictions = yfin_pred.to_csv('predictions.csv', sep= '\t')

In [40]:
#json file using pandas directly
predictions = yfin_pred.to_json('predictions.json')